In [1]:
import os
os.environ['OMP_NUM_THREADS'] = "16"
os.environ['KMP_AFFINITY'] = "granularity=fine,compact,1,0"
os.environ['KMP_BLOCKTIME'] = "1"
os.environ['OMP_DISPLAY_ENV'] = "True"

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity


OPENMP DISPLAY ENVIRONMENT BEGIN
  _OPENMP = '201511'
  OMP_DYNAMIC = 'FALSE'
  OMP_NESTED = 'FALSE'
  OMP_NUM_THREADS = '16'
  OMP_SCHEDULE = 'DYNAMIC'
  OMP_PROC_BIND = 'FALSE'
  OMP_PLACES = ''
  OMP_STACKSIZE = '22512930916562'
  OMP_WAIT_POLICY = 'PASSIVE'
  OMP_THREAD_LIMIT = '4294967295'
  OMP_MAX_ACTIVE_LEVELS = '2147483647'
  OMP_CANCELLATION = 'FALSE'
  OMP_DEFAULT_DEVICE = '0'
  OMP_MAX_TASK_PRIORITY = '0'
OPENMP DISPLAY ENVIRONMENT END

OPENMP DISPLAY ENVIRONMENT BEGIN
   _OPENMP='201611'
  [host] OMP_AFFINITY_FORMAT='OMP: pid %P tid %i thread %n bound to OS proc set {%A}'
  [host] OMP_ALLOCATOR='omp_default_mem_alloc'
  [host] OMP_CANCELLATION='FALSE'
  [host] OMP_DEBUG='disabled'
  [host] OMP_DEFAULT_DEVICE='0'
  [host] OMP_DISPLAY_AFFINITY='FALSE'
  [host] OMP_DISPLAY_ENV='TRUE'
  [host] OMP_DYNAMIC='FALSE'
  [host] OMP_MAX_ACTIVE_LEVELS='1'
  [host] OMP_MAX_TASK_PRIORITY='0'
  [host] OMP_NESTED: deprecated; max-active-levels-var=1
  [host] OMP_NUM_TEAMS='0'
  [host] OM

In [3]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):

    transform = transforms.Compose([transforms.ToTensor()])
    
    train_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    valid_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)
  
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, valid_loader, test_loader)

In [4]:
def train(model, data_loader, valid_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            #if (step % 100 == 0):
            #    print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
               
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            

In [5]:
def validate(model, valid_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
    print('Accuracy of the network on the {} validation images: {:.2f} %'.format(5000, 100 * correct / total)) 

In [6]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))  

In [7]:
def count_parameters(model):
    parameters = list(model.parameters())
    total_parms = sum([np.prod(p.size()) for p in parameters if p.requires_grad])
    return total_parms


In [8]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.fc1 = nn.Linear(64*7*7, 512, bias=False)
        
        self.fc2 = nn.Linear(512, num_classes, bias=False)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [9]:
# General parameters
data_dir = '/tmp'
device = 'cpu'
num_classes = 10

# Hyperparameters
max_lr = 0.00001
weight_decay = 0.005
batch_size = 256
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# FashionMNIST dataset 
train_loader, valid_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

#model
model = CNN(num_classes)
print(count_parameters(model))

# Optimizer
optimizer = optimizer(model.parameters(), max_lr, weight_decay=weight_decay)

1629472


In [10]:
%%time
train(model, train_loader, valid_loader, num_epochs, criterion, optimizer, device)

Epoch [1/10], Loss: 1.5469
Epoch [2/10], Loss: 1.0422
Epoch [3/10], Loss: 0.7749
Epoch [4/10], Loss: 0.7135
Epoch [5/10], Loss: 0.6508
Epoch [6/10], Loss: 0.5759
Epoch [7/10], Loss: 0.6090
Epoch [8/10], Loss: 0.6122
Epoch [9/10], Loss: 0.5313
Epoch [10/10], Loss: 0.5372
CPU times: user 11min 30s, sys: 3min 24s, total: 14min 54s
Wall time: 1min 25s


In [15]:
test(model, test_loader, device)

Accuracy of the network on the 10000 test images: 78.46 %


In [12]:
schedule = torch.profiler.schedule(wait=5, warmup=5, active=3)

#prof = profile(schedule=schedule, activities=[ProfilerActivity.CPU], with_stack=True, record_shapes=True, on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/cnn/'))
prof = profile(activities=[ProfilerActivity.CPU])
        
input_sample, _ = next(iter(train_loader))

prof.start()
model(input_sample)
prof.stop()

#prof.export_chrome_trace("./cnn_trace.json")
print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::mkldnn_convolution        51.07%       5.597ms        51.31%       5.624ms       2.812ms             2  
    aten::max_pool2d_with_indices        26.53%       2.908ms        26.53%       2.908ms       1.454ms             2  
                  aten::clamp_min        14.36%       1.574ms        14.36%       1.574ms     787.000us             2  
                         aten::mm         5.87%     643.000us         5.87%     643.000us     321.500us             2  
                     aten::linear         0.39%      43.000us         6.17%     676.000us     338.000us             2  
                aten::convolution       

STAGE:2023-05-14 12:35:15 2963507:2963507 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-05-14 12:35:15 2963507:2963507 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-14 12:35:15 2963507:2963507 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
